In [2]:
import numpy as np
import json
from utils import preprocess
from tqdm.notebook import trange, tqdm


## Load data to Index

In [3]:
f = open('ViQuAD/train_ViQuAD.json', 'r')
train_set = json.load(f)
train_set = train_set['data']
f = open('ViQuAD/dev_ViQuAD.json', 'r')
dev_set = json.load(f)
dev_set = dev_set['data']
f = open('ViQuAD/test_ViQuAD.json', 'r')
test_set = json.load(f)
test_set = test_set['data']

In [4]:
contexts = []
questions = []
id = 0
all_data = train_set+dev_set+test_set
for article in all_data:
    title = article['title']
    for paragraph in article['paragraphs']:
        context = preprocess(paragraph['context'])
        contexts.append(
            {'id': id,
            'contents': context,
            'title': title
        })
        for q in paragraph['qas']:
            questions.append({'answers': context,
                             'question': q['question']})
        id+=1
len(contexts)
# with open('collections/documents.json','w') as f:
#     json.dump(contexts, f)


5109

In [7]:
import pandas as pd
df_wiki_windows = pd.read_csv("/mmlabworkspace/Students/AIC/old/zalo/hungcv/temp/wikipedia_20220620_cleaned_v2.csv")
df_wiki = pd.read_csv("/mmlabworkspace/Students/AIC/old/zalo/hungcv/data/wikipedia_20220620_short.csv")
df_wiki.title = df_wiki.title.apply(str)


In [17]:
corpus = df_wiki_windows['bm25_text'].values

In [18]:
f=open('./doc2query/doc2query.json')
data = json.load(f)

id = 5109
for i in corpus:
    data.append(
        {'id': id,
        'contents': i,
        'title': 'wiki',
        'ori_contents': i

    })

    id+=1

In [22]:
data[0]

{'id': 0,
 'contents': 'phạm văn đồng ( 1 tháng 3 năm 1906 – 29 tháng 4 năm 2000 ) là thủ_tướng đầu_tiên của nước cộng_hòa xã_hội chủ_nghĩa việt_nam từ năm 1976 ( từ năm 1981 gọi là chủ_tịch hội_đồng_bộ_trưởng ) cho đến khi nghỉ hưu năm 1987 . trước đó ông từng giữ chức_vụ thủ_tướng chính_phủ việt nam dân_chủ cộng_hòa từ năm 1955 đến năm 1976 . ông là vị thủ_tướng việt_nam tại vị lâu nhất ( 1955 – 1987 ) . ông là học_trò , cộng_sự của chủ_tịch hồ chí minh . ông có tên gọi thân_mật là tô , đây từng là bí_danh của ông . ông còn có tên gọi là lâm bá kiệt khi làm phó chủ_nhiệm cơ_quan biện sự xứ tại quế lâm ( chủ_nhiệm là hồ học lãm ) . tên của thủ_tướng việt nam là gì ai là chủ_tịch chính_phủ đầu_tiên của nước cộng_hòa xã_hội chủ_nghĩa việt nam thủ_tướng việt nam là ai thủ_tướng việt nam là ai phạm văn đồng là ai',
 'title': 'Phạm Văn Đồng',
 'ori_contents': 'phạm văn đồng ( 1 tháng 3 năm 1906 – 29 tháng 4 năm 2000 ) là thủ_tướng đầu_tiên của nước cộng_hòa xã_hội chủ_nghĩa việt_nam từ năm

In [21]:
!mkdir full_wiki
with open('full_wiki/documents.json','w') as f:
    json.dump(data, f)

mkdir: cannot create directory ‘full_wiki’: File exists


## Index

In [ ]:
!python -m pyserini.index.lucene \
    --collection JsonCollection \
    --input full_wiki \
    --index indexes/full_wiki \
    --generator DefaultLuceneDocumentGenerator \
    --threads 4 \
    --storePositions --storeDocvectors --storeRaw

## test searcher

In [33]:
from retrieval import Pyserini_Search
searcher = Pyserini_Search(index_path='indexes/full_wiki')
# searcher.search("Cộng hòa Weimar chính thức thay thế đế quốc Đức kể từ sau sự kiện nào?", 100)

Some weights of the model checkpoint at nguyenvulebinh/vi-mrc-base were not used when initializing RobertaModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Evaluate

In [34]:
## Read data from test_set
questions = []
id = 0
for article in test_set:
    title = article['title']
    for paragraph in article['paragraphs']:
        for q in paragraph['qas']:
            questions.append({'id': q['id'],
                              'gt_contexts': preprocess(paragraph['context']),
                             'question': q['question'],
                             'answers': q['answers'],
                             'title': title
                             })
        id+=1

for i in trange(len(questions)):
    contexts, ranking_scores = searcher.search(questions[i]['question'],use_rerank=True, topk=100)
    questions[i]['contexts'] = contexts

In [35]:
def evaluate(questions, topk=[1,5,10], max_k=100):
    accuracy = { k : [] for k in topk }
    out = []
    for question in tqdm(questions):
        answers = question['gt_contexts']    #ddasp an
        contexts = question['contexts']  #search ra
        has_ans_idx = max_k  # first index in contexts that has answers
        for idx, ctx in enumerate(contexts):
            if idx >= max_k:
                break
            if ctx==answers:
                has_ans_idx = idx
                out.append(has_ans_idx)
                break
        for k in topk:
            accuracy[k].append(0 if has_ans_idx >= k else 1)
            
    for k in topk:
        print(f'Top{k}\taccuracy: {np.mean(accuracy[k]):.4f}')


In [36]:
evaluate(questions, topk=[1,5,10, 15, 20, 25, 30], max_k=100)


Top1	accuracy: 0.6516
Top5	accuracy: 0.8213
Top10	accuracy: 0.8357
Top15	accuracy: 0.8575
Top20	accuracy: 0.8787
Top25	accuracy: 0.8937
Top30	accuracy: 0.9023


## Create ranking dataset

In [51]:
len(train_set+dev_set)

138

In [42]:
questions = []
id = 0
all_data = train_set+dev_set+test_set
for article in all_data:
    title = article['title']
    for paragraph in article['paragraphs']:
        for q in paragraph['qas']:
            questions.append({'answers': preprocess(paragraph['context']),
                            'question': q['question']})
        id+=1

from tqdm.notebook import trange, tqdm
for i in trange(len(questions)):
    hits = searcher.search(preprocess(questions[i]['question']),k=100)
    contexts = []
    questions[i]['contexts'] = [json.loads(hits[j].raw)['contents'] for j in range(min(len(hits),100))]


In [63]:
texts = []
queries = []
labels = []
data_types= []

test_index = 0
for article in (train_set+dev_set):
    for paragraph in article['paragraphs']:
        test_index += len(paragraph['qas'])

for index, question in tqdm(enumerate(questions)):
    neg, pos = retrievaler.sample( question['question'], question['answers'], 11, max_query_len = 512)
    a = question['question']

    # assert len(neg) == 50, f'len(neg)={len(neg)} should be 50'
    samples = len(neg) +1
    
    texts.append(pos)
    labels.append(1)    
    texts.extend(neg)
    labels.extend([0]*(samples-1))
    queries.extend([question['question']]*samples)

    if index < test_index:
        data_types.extend(['train']*samples)
    else:
        data_types.extend(['test']*samples)

In [64]:
import pandas as pd
df = pd.DataFrame()
df["text"] = texts
df["queries"] = queries
df["label"] = labels
df['data_types'] = data_types

In [68]:
df[df.data_types=='train']

,text,queries,label,data_types
0,phạm văn đồng ( 1 tháng 3 năm 1906 – 29 tháng ...,Tên gọi nào được Phạm Văn Đồng sử dụng khi làm...,1,train
1,phạm văn đồng có vợ là bà phạm thị cúc và một ...,Tên gọi nào được Phạm Văn Đồng sử dụng khi làm...,0,train
2,"ông việt phương , nguyên thư_ký của thủ_tướng ...",Tên gọi nào được Phạm Văn Đồng sử dụng khi làm...,0,train
3,bình định là mảnh đất có bề dày lịch_sử với nề...,Tên gọi nào được Phạm Văn Đồng sử dụng khi làm...,0,train
4,"đầu năm 1126 , triều_kim của người nữ chân đã ...",Tên gọi nào được Phạm Văn Đồng sử dụng khi làm...,0,train
...,...,...,...,...
229479,"uganda tuyên_bố đi theo đường_lối trung_lập , ...","Hiện nay, kinh tế Uganda cần phải tập trung ph...",0,train
229480,"từ ngàn năm nay , việt nam là một nước nông_ng...","Hiện nay, kinh tế Uganda cần phải tập trung ph...",0,train
229481,hoạt_động_kinh_tế ở khu_vực paris cũng đa_dạng...,"Hiện nay, kinh tế Uganda cần phải tập trung ph...",0,train
229482,bất_kỳ tác vụ nào có_thể phá_hủy dữ_liệu của m...,"Hiện nay, kinh tế Uganda cần phải tập trung ph...",0,train


In [66]:
df.to_csv('train_ranking.csv',index=False)

In [67]:
!head train_ranking.csv

text,queries,label,data_types
"phạm văn đồng ( 1 tháng 3 năm 1906 – 29 tháng 4 năm 2000 ) là thủ_tướng đầu_tiên của nước cộng_hòa xã_hội chủ_nghĩa việt_nam từ năm 1976 ( từ năm 1981 gọi là chủ_tịch hội_đồng_bộ_trưởng ) cho đến khi nghỉ hưu năm 1987 . trước đó ông từng giữ chức_vụ thủ_tướng chính_phủ việt nam dân_chủ cộng_hòa từ năm 1955 đến năm 1976 . ông là vị thủ_tướng việt_nam tại vị lâu nhất ( 1955 – 1987 ) . ông là học_trò , cộng_sự của chủ_tịch hồ chí minh . ông có tên gọi thân_mật là tô , đây từng là bí_danh của ông . ông còn có tên gọi là lâm bá kiệt khi làm phó chủ_nhiệm cơ_quan biện sự xứ tại quế lâm ( chủ_nhiệm là hồ học lãm ) .",Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm?,1,train
"phạm văn đồng có vợ là bà phạm thị cúc và một người con_trai duy_nhất tên là phạm sơn_dương , hiện là thiếu_tướng quân_đội nhân_dân việt_nam , phó_giám_đốc viện khoa_học và công_nghệ quân_sự . sau khi lấy bà cúc ( tháng 10 năm 1946 ) phạm văn đồng v

## Test Ranking

In [5]:
from model import *
pairwise_model_stage1 = PairwiseModel("nguyenvulebinh/vi-mrc-base", )
pairwise_model_stage1.load_state_dict(torch.load("pairwise_v2.bin"))
pairwise_model_stage1.eval()
pairwise_model_stage1.to('cuda')

Some weights of the model checkpoint at nguyenvulebinh/vi-mrc-base were not used when initializing RobertaModel: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at nguyenvulebinh/vi-mrc-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PairwiseModel(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768

In [6]:
#Reranking with pairwise model for top10
def rerank(question, contexts):
    ranking_preds = pairwise_model_stage1.stage1_ranking(question, contexts)
    ranking_scores = ranking_preds 

    #Question answering
    best_idxs = np.argsort(ranking_scores)[-10:]
    ranking_scores = np.array(ranking_scores)[best_idxs]
    texts = np.array(contexts)[best_idxs]
    return texts 